## Functions and imports

In [13]:
from pathlib import Path
import pandas as pd
import time


def get_df(dir: str) -> pd.DataFrame:
    """Create a DataFrame from downloaded vox celeb txt files.
    Dataframe contains following columns:
    speaker: speaker id,
    video: video id,
    txt_file: txt file name,
    start_time: start frame of the video,
    end_time: end frame of the video,
    length: length of the video in seconds.

    Args:
        dir (str): path to txt folder
    """
    data = []
    for speaker_dir in Path(dir).iterdir():
        if not speaker_dir.is_dir():
            continue
        for video_dir in speaker_dir.iterdir():
            if not video_dir.is_dir():
                continue
            for txt_file in video_dir.iterdir():

                with open(txt_file, "r") as f:
                    text = f.read()
                # read 8. line
                start_text = text.split("\n")[7]
                # get start time
                start_time = start_text.split(" ")[0]
                start_time = int(start_time)
                # get end time from last line
                end_time = text.split("\n")[-2].split(" ")[0]
                end_time = int(end_time)
                data.append(
                    (
                        speaker_dir.name,
                        video_dir.name,
                        txt_file.name,
                        start_time,
                        end_time,
                    )
                )

    df = pd.DataFrame(
        data, columns=["speaker", "video", "txt_file", "start_time", "end_time"]
    )
    df["length"] = (df["end_time"] - df["start_time"]) / 25

    # sort by speaker and length
    return df.sort_values(by=["speaker", "length"], ascending=[True, False])


def get_df_by_downloaded_folder(folder: str) -> pd.DataFrame:
    """Create a DataFrame from folder with downloaded mp3 files.

    Args:
        folder (str): path to folder with downloaded mp3 files.
    """
    # iterate all files in the folder and get their length
    data = []

    for file in Path(folder).rglob("*.mp3"):
        try:
            user_id = file.stem.split("_")[0]  # user_id in voxceleb2
            duration_ms = int(file.stem.split("_")[2])  # duration in ms
            # Convert to seconds
            duration_s = duration_ms / 1000.0
            data.append({"speaker": user_id, "duration_s": duration_s})
        except Exception as e:
            #print(f"Error processing file {file}: {e}")
            continue
    return pd.DataFrame(data)

In [2]:
import yt_dlp
from pydub import AudioSegment


def download_audio(youtube_url, output_path="audio.mp3"):
    """Download audio from a YouTube video and save it to a file."""
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": output_path,
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
                "preferredquality": "192",
            }
        ],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

    print(f"Audio downloaded as {output_path}")


def check_video_available(youtube_url):
    """Check if a YouTube video is available."""
    ydl_opts = {
        "quiet": True,
        "no_warnings": True,
        "format": "bestaudio/best",
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(youtube_url, download=False)
            return True
    except yt_dlp.utils.DownloadError as e:
        return
    return info is not None


def trim_audio(input_path, output_path, start_time, end_time):
    """Trim an audio file between start_time and end_time (in seconds)."""

    # Load the audio file
    audio = AudioSegment.from_file(input_path)

    # Convert times to milliseconds
    trimmed_audio = audio[start_time * 1000 : end_time * 1000]

    # Export the trimmed audio
    trimmed_audio.export(output_path, format="mp3")
    print(f"Trimmed audio saved as {output_path}")

In [3]:
def download_df_audio(
    df: pd.DataFrame, out_dir: str, n_samples: int = 2, start_index: int = 0
):
    """Download audio files from youtube urls in the DataFrame.
    Naming convention for the output files: {speaker}_{sample_number}_{duration}_{video}_{txt_file}_cut.mp3
    DataFrame should contain columns: speaker, video, txt_file, start_time, end_time.

    Args:
        df (pd.DataFrame): DataFrame containing the data.
        out_dir (str): Output directory for
        n_samples (int): Number of samples to download for each speaker.
        start_index (int): Start naming index for the output files.
    """

    active_speaker = None
    """Variable to keep track of the current speaker."""
    downloaded_samples = 0
    """Variable to keep track of the number of samples downloaded for the current speaker."""
    last_video = None
    """Variable to keep track of the last video downloaded for the current speaker."""

    Path(out_dir).mkdir(parents=True, exist_ok=True)

    # sort by speaker and length
    df = df.sort_values(by=["speaker", "length"], ascending=[True, False])

    for index, row in df.iterrows():
        speaker = row["speaker"]
        video = row["video"]
        txt_file = row["txt_file"].replace(".txt", "")
        start_time = row["start_time"]  # in frames
        end_time = row["end_time"]  # in frames
        duration = int(row["length"] * 1000)  # in milliseconds

        if active_speaker != speaker:
            active_speaker = speaker
            downloaded_samples = 0

        # skip if the same video for the same speaker
        if last_video == video:
            continue

        # skip if already downloaded 2 samples
        if downloaded_samples >= n_samples:
            continue

        print(f"Speaker: {speaker}, Video: {video}, Txt file: {txt_file}")

        # get youtube url
        youtube_url = f"https://www.youtube.com/watch?v={video}"
        output_name = (
            f"{speaker}_{downloaded_samples+start_index}_{duration}_{video}_{txt_file}"
        )
        output_path = f"{output_name}.mp3"

        if not check_video_available(youtube_url):
            continue

        # download audio
        for i in range(3):
            try:
                download_audio(youtube_url, output_name)
                break
            except Exception as e:
                print(f"Failed to download audio: {e}")
                # wait for 5 seconds
                time.sleep(5)
            if i == 2:
                raise Exception("Failed to download audio after 3 attempts")

        # cut the audio
        # !ffmpeg -i {output_path} -ss {start_time} -to {end_time} -c copy {speaker}_{video}_cut.mp3
        trim_audio(
            output_path,
            f"{out_dir}/{output_name}_cut.mp3",
            start_time / 25,
            end_time / 25,
        )

        Path(output_path).unlink()
        downloaded_samples += 1
        last_video = video

## Download txt data from the web

https://mm.kaist.ac.kr/datasets/voxceleb/index.html#testlist

download test data

```bash
curl -O https://mm.kaist.ac.kr/datasets/voxceleb/data/vox2_test_txt.zip
mkdir -p ../data/test
unzip vox2_test_txt.zip -d ../data/test
```

download dev data

```bash
curl -O https://mm.kaist.ac.kr/datasets/voxceleb/data/vox2_dev_txt.zip
mkdir -p ../data/dev
unzip vox2_dev_txt.zip -d ../data/dev
```

## Test dataset

In [ ]:
IN_FOLDER = "../data/test/txt"

df_test = get_df(IN_FOLDER)
df_test

In [ ]:
df_test.to_csv("../data/test.csv", index=False)

In [ ]:
download_df_audio(df_test, "../data/test_data")

# Dev dataset



## Create dev dataframe

In [ ]:
df_dev = get_df("../data/dev/txt/")
df_dev

In [ ]:
df_dev.to_csv("../data/dev.csv", index=False)

## Download Most Famous Celebrities

In [4]:
import pandas as pd

NUM_OF_SAMPLES = 8
START_INDEX = 2
NUM_OF_CELEBS = 300

In [21]:
df_meta = pd.read_csv("../data/vox2_meta.csv")

# get NUM_OF_CELEBS celebs from dev set
df_meta = df_meta[df_meta["Set"] == "dev"]
df_meta = df_meta.sort_values(by="wiki_views_2024", ascending=False).head(NUM_OF_CELEBS)
df_meta

,Name,VoxCeleb2_ID,VGGFace2_ID,Gender,Set,wiki_views_2024
3031,Kamala_Harris,id04538,n004537,f,dev,30250192.0
4051,Mike_Tyson,id06121,n006120,m,dev,11900865.0
761,Bob_Marley,id01109,n001109,m,dev,10636588.0
3974,Michael_Jackson,id06004,n006003,m,dev,9373493.0
504,Ariana_Grande,id00752,n000752,f,dev,9362854.0
...,...,...,...,...,...,...
3100,Katherine_Schwarzenegger,id04642,n004641,f,dev,1572645.0
4475,Patrick_J._Adams,id06813,n006812,m,dev,1569381.0
1235,Cole_Sprouse,id01772,n001771,m,dev,1564609.0
3096,Katharine_Ross,id04638,n004637,f,dev,1560600.0


In [22]:
df_dev = pd.read_csv("../data/dev.csv")

# limit df_dev to celebs in df_meta
df_top_celebs = df_dev[df_dev["speaker"].isin(df_meta["VoxCeleb2_ID"])]
df_top_celebs

,speaker,video,txt_file,start_time,end_time,length
1235,id00020,iRsViE6yAeg,00307.txt,1592,2053,18.44
1236,id00020,iRsViE6yAeg,00311.txt,3284,3741,18.28
1237,id00020,IpLYBI5Kaow,00064.txt,4414,4821,16.28
1238,id00020,l9PiLBUrSGs,00354.txt,8348,8750,16.08
1239,id00020,MM1fE2IkaDQ,00096.txt,2292,2648,14.24
...,...,...,...,...,...,...
1082801,id09185,AoO24VZf4oM,00069.txt,2827,2927,4.00
1082802,id09185,NDKxz17BhdI,00082.txt,6795,6895,4.00
1082803,id09185,nHM0sPZFkXQ,00154.txt,618,717,3.96
1082804,id09185,5TZfJJFAOwo,00031.txt,11828,11927,3.96


In [39]:
# remove all rows with longer length than already downloaded samples

already_downloaded = get_df_by_downloaded_folder("../data/download_df_audio/")
already_downloaded = already_downloaded.sort_values(by="speaker")
# get min length for each speaker
already_downloaded = already_downloaded.groupby("speaker").min().reset_index()

#Sort out the speakers rows rom df_top_celebs which are present in already_downloaded
ids = set(already_downloaded["speaker"].astype(str).str.strip())
filtered_df = df_top_celebs[~df_top_celebs["speaker"].astype(str).str.strip().isin(ids)]

df_top_celebs_filtered = df_top_celebs.merge(
    already_downloaded, on="speaker", how="left", suffixes=("", "_y")
)
df_top_celebs_filtered = df_top_celebs_filtered[
    df_top_celebs_filtered["length"] < df_top_celebs_filtered["duration_s"]
]



In [40]:
download_df_audio(
    filtered_df,
    "../data/download_df_audio",
    n_samples=NUM_OF_SAMPLES,
    start_index=START_INDEX,
)

Speaker: id08167, Video: GXb9-3UCtLQ, Txt file: 00140


ERROR: [youtube] GXb9-3UCtLQ: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.


Speaker: id08167, Video: GXb9-3UCtLQ, Txt file: 00119


ERROR: [youtube] GXb9-3UCtLQ: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.


Speaker: id08167, Video: YvEmSk1Tm9M, Txt file: 00207
[youtube] Extracting URL: https://www.youtube.com/watch?v=YvEmSk1Tm9M
[youtube] YvEmSk1Tm9M: Downloading webpage
[youtube] YvEmSk1Tm9M: Downloading tv client config
[youtube] YvEmSk1Tm9M: Downloading player 5ae7d525
[youtube] YvEmSk1Tm9M: Downloading tv player API JSON
[youtube] YvEmSk1Tm9M: Downloading ios player API JSON
[youtube] YvEmSk1Tm9M: Downloading m3u8 information
[info] YvEmSk1Tm9M: Downloading 1 format(s): 140
[download] Destination: id08167_2_16120_YvEmSk1Tm9M_00207
[download] 100% of    8.52MiB in 00:00:01 at 6.74MiB/s   
[FixupM4a] Correcting container of "id08167_2_16120_YvEmSk1Tm9M_00207"
[ExtractAudio] Destination: id08167_2_16120_YvEmSk1Tm9M_00207.mp3
Deleting original file id08167_2_16120_YvEmSk1Tm9M_00207 (pass -k to keep)
Audio downloaded as id08167_2_16120_YvEmSk1Tm9M_00207
Trimmed audio saved as ../data/download_df_audio/id08167_2_16120_YvEmSk1Tm9M_00207_cut.mp3
Speaker: id08167, Video: Htafo8uMxW8, Txt file:

...

Speaker: id09185, Video: 5TZfJJFAOwo, Txt file: 00041
[youtube] Extracting URL: https://www.youtube.com/watch?v=5TZfJJFAOwo
[youtube] 5TZfJJFAOwo: Downloading webpage
[youtube] 5TZfJJFAOwo: Downloading tv client config
[youtube] 5TZfJJFAOwo: Downloading player 5ae7d525
[youtube] 5TZfJJFAOwo: Downloading tv player API JSON
[youtube] 5TZfJJFAOwo: Downloading ios player API JSON
[youtube] 5TZfJJFAOwo: Downloading m3u8 information
[info] 5TZfJJFAOwo: Downloading 1 format(s): 251
[download] Destination: id09185_7_10800_5TZfJJFAOwo_00041
[download] 100% of   27.22MiB in 00:00:17 at 1.59MiB/s     
[ExtractAudio] Destination: id09185_7_10800_5TZfJJFAOwo_00041.mp3
Deleting original file id09185_7_10800_5TZfJJFAOwo_00041 (pass -k to keep)
Audio downloaded as id09185_7_10800_5TZfJJFAOwo_00041
Trimmed audio saved as ../data/download_df_audio/id09185_7_10800_5TZfJJFAOwo_00041_cut.mp3
Speaker: id09185, Video: 17oel-iAWAQ, Txt file: 00025


ERROR: [youtube] 17oel-iAWAQ: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.


Speaker: id09185, Video: XYhArFXKUeY, Txt file: 00106


ERROR: [youtube] XYhArFXKUeY: Video unavailable


Speaker: id09185, Video: _cDhsiYaV3c, Txt file: 00111
[youtube] Extracting URL: https://www.youtube.com/watch?v=_cDhsiYaV3c
[youtube] _cDhsiYaV3c: Downloading webpage
[youtube] _cDhsiYaV3c: Downloading tv client config
[youtube] _cDhsiYaV3c: Downloading player 5ae7d525
[youtube] _cDhsiYaV3c: Downloading tv player API JSON
[youtube] _cDhsiYaV3c: Downloading ios player API JSON
[youtube] _cDhsiYaV3c: Downloading m3u8 information
[info] _cDhsiYaV3c: Downloading 1 format(s): 251
[download] Destination: id09185_8_10320__cDhsiYaV3c_00111
[download] 100% of    4.75MiB in 00:00:00 at 13.85MiB/s  
[ExtractAudio] Destination: id09185_8_10320__cDhsiYaV3c_00111.mp3
Deleting original file id09185_8_10320__cDhsiYaV3c_00111 (pass -k to keep)
Audio downloaded as id09185_8_10320__cDhsiYaV3c_00111
Trimmed audio saved as ../data/download_df_audio/id09185_8_10320__cDhsiYaV3c_00111_cut.mp3
Speaker: id09185, Video: AvMgRNndnoA, Txt file: 00072
[youtube] Extracting URL: https://www.youtube.com/watch?v=AvMgRN

## Download all Dev audio files

In [ ]:
import pandas as pd

df = pd.read_csv("../data/dev.csv")
df = df.sort_values(by=["speaker", "length"], ascending=[True, False])
# split df by speaker id to batches of 100 ids

# Get unique speakers
unique_speakers = df["speaker"].unique()

# Define batch size
batch_size = 100

# Create batches
batches = [
    unique_speakers[i : i + batch_size]
    for i in range(0, len(unique_speakers), batch_size)
]

# Create a DataFrame for each batch
batch_dfs = [df[df["speaker"].isin(batch)] for batch in batches]

In [ ]:
# TODO set start_batch
start_batch = 7
for i, batch_df in enumerate(batch_dfs):
    if i < start_batch:
        continue

    out_dir = f"../data/dev_data/batch_{i}"
    download_df_audio(batch_df, out_dir)